In [21]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import random
import re
import os
from os.path import join
import math
import time
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import seaborn as sns
from langdetect import detect
from langcodes import *
import language_data
from collections import Counter

from nltk import word_tokenize,sent_tokenize,pos_tag
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
import string
stopword_list = stopwords.words('english')
stopword_list.extend(['\'s','\'nt','’','``',"n't",'·'])
for mark in string.punctuation:
    stopword_list.append(mark)
    
greek_users = pd.read_csv('greek_users.tsv',sep='\t')
dutch_users = pd.read_csv('dutch_users.tsv',sep='\t')

greek_users = pd.read_csv('greek_users.tsv',sep='\t')
greek_users = greek_users.query( 'nr_reviews<1000' ).iloc[:200]
dutch_users = pd.read_csv('dutch_users.tsv',sep='\t')
dutch_users = dutch_users.query( 'nr_reviews<1000' ).iloc[:200]


users_dir = 'Users'

data = []

nationality=''

for i,row in pd.concat([dutch_users, greek_users]).iterrows():
    user = []
    user.append(row["id"])
    
    if i==0 and nationality=='Dutch':
        nationality='Greek'
    elif i==0:
        nationality='Dutch'
    user.append(nationality)
    xml_file = join(users_dir,f'{row["id"]}.xml')
    tree = ET.parse(xml_file)
    root = tree.getroot()
    nr_reviews = root.find('user/reviews_count')
    if nr_reviews is not None:
        user.append(int(nr_reviews.text))
    nr_friends = root.find('user/friends_count')
    if nr_friends is not None:
        user.append(int(nr_friends.text))
    data.append(user)
    
df_users = pd.DataFrame(data,columns=['id','nationality','reviews','friends'])

df_users = df_users.query('reviews<1000') 

In [1]:
avg_rating_dict = dict()

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
]


def select_agent():
    n=random.randint(0,len(user_agents)-1)
    return user_agents[n]

user_dir = 'Users_data'
if not os.path.exists(user_dir):
    os.makedirs(user_dir)
    
print(df_users.shape)

for i,row in df_users.iterrows():
    
    if not os.path.isfile(os.path.join(user_dir,f'{row["id"]}.txt')):

        per_page = 20
        nr_pages = 1

        user_id = row['id']
        print(user_id)

        reviews = []
        base_url = 'https://www.goodreads.com/review/list/'
        
        url = f'{base_url}{user_id}?utf8=%E2%9C%93&order=d&sort=review&page=1&view=reviews&per_page={per_page}'
        print(url)
        
        headers = {'User-Agent': select_agent() }
        response = requests.get( url, headers=headers)

        if response:
            html_page = response.text
            soup = BeautifulSoup(html_page,"lxml")

            links = soup.findAll('a')
            for a in links:
                href = a.get('href')
                if href is not None:
                    if re.search(r'shelf=%23ALL%23',str(href)):
                        nr_reviews = re.sub(r'\D','',a.text)

                        if re.search(r'\d',nr_reviews):
                            nr_reviews = int(nr_reviews.strip())
                            nr_pages = math.ceil(nr_reviews/per_page)
                            break
                    
        rating = 0
        author_links = []
        
        for i in range(1,(nr_pages+1)):

            url = f'{base_url}{user_id}?utf8=%E2%9C%93&order=d&sort=review&page={i}&view=reviews&per_page={per_page}'
            #print(url)

            headers = {'User-Agent': select_agent() }
            response = requests.get( url, headers=headers)

            if response:
                html_page = response.text
                soup = BeautifulSoup(html_page,"lxml")
                stars = soup.findAll('span',{'class':'staticStar'})
                
                for star in stars:
                    if 'p10' in star.get('class'):
                        rating += 1
                        
                        
                
                links = soup.findAll('a')
                for a in links:
                    if re.search( 'author/show/' , str(a.get('href')) ):
                        #print(a.get('href'))
                        author_links.append(str(a.get('href')))
                
            
        avg_rating = rating / nr_reviews
        print(avg_rating)
        with open(os.path.join(user_dir,f'{user_id}.txt'),'w') as out:
            out.write(f'{avg_rating}\n')
            out.write(f'{nr_reviews}\n\n')
            for a in author_links:
                out.write(f'{a}\n')

        time.sleep(5)        
        

(606, 4)
94036318
https://www.goodreads.com/review/list/94036318?utf8=%E2%9C%93&order=d&sort=review&page=1&view=reviews&per_page=20


KeyboardInterrupt: 

In [27]:
user_dir = 'Users_data'

data = []
print(df_users.shape)

for i,row in df_users.iterrows():

    line = dict()
    line['id'] = row['id']
    line['nationality'] = row['nationality']
    line['reviews'] = row['reviews']
    try:
        file = open(os.path.join(user_dir,f"{row['id']}.txt"))
        contents = file.readlines()
        line['average_rating'] = float(contents[0].strip())
    except:
        line['average_rating'] = None
    data.append(line)
    

df_rating = pd.DataFrame(data)         

(400, 4)


In [29]:
df_rating.to_csv('average_rating.csv')

In [28]:
df_rating.shape

(400, 4)

In [54]:
user_dir = 'Users_data'

dutch_authors = []
greek_authors = []

for i,row in df_users.iterrows():

    line = dict()
    line['id'] = row['id']
    line['nationality'] = row['nationality']

    try:
        file = open(os.path.join(user_dir,f"{row['id']}.txt"))
        contents = file.readlines()
        contents = contents[3:]
        for author in contents:
            if line['nationality'] == 'Dutch':
                dutch_authors.append(author.strip())
            else:
                greek_authors.append(author.strip())
                      
    except:
        print('Issue with ' + str(line['id']))

dutch_authors = set(list(dutch_authors))
greek_authors = set(list(greek_authors)) 
dutch_authors = [f'https://www.goodreads.com{author}' for author in dutch_authors]
greek_authors = [f'https://www.goodreads.com{author}' for author in greek_authors]

In [56]:
with open('authors_greek.txt','w',encoding='utf-8') as out:
    for a in greek_authors:
        out.write(f'{a}\n')

        
with open('authors_dutch.txt','w',encoding='utf-8') as out:
    for a in dutch_authors:
        out.write(f'{a}\n')
